In [1]:
import numpy as np
import mido
from keras.models import Sequential
from keras.layers import Activation,LSTM,Dropout,Flatten,Dense
import os

Using TensorFlow backend.


# DATA LOADING

In [2]:
x_train = np.load('/home/ritik/Desktop/music_generation_model/x_train.npy')
y_train = np.load('/home/ritik/Desktop/music_generation_model/y_train.npy')


In [3]:
print(x_train.shape, y_train.shape)

(21867, 3, 4) (21867, 4)


In [4]:
model = Sequential()

model.add(LSTM(128, input_shape = (3, 4), return_sequences= True))
model.add(Dropout(0.3))
model.add(LSTM(64, return_sequences= False))
model.add(Dropout(0.2))
#model.add(LSTM(16, return_sequences=False))
model.add(Dense(4))
model.add(Activation('linear'))
model.compile(loss = 'categorical_crossentropy', optimizer = 'RMSprop', metrics = ['accuracy'])
model.summary() 

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 3, 128)            68096     
_________________________________________________________________
dropout_1 (Dropout)          (None, 3, 128)            0         
_________________________________________________________________
lstm_2 (LSTM)                (None, 64)                49408     
_________________________________________________________________
dropout_2 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 4)                 260       
_________________________________________________________________
activation_1 (Activation)    (None, 4)                 0         
Total params: 117,764
Trainable params: 117,764
Non-trainable params: 0
_________________________________________________________________


In [5]:
model.fit(x_train, y_train, epochs=20, batch_size=60, verbose = 1)

Epoch 1/20
21867/21867 [==============================] - 6s 273us/step - loss: -5.1117 - acc: 0.2072
Epoch 2/20
21867/21867 [==============================] - 5s 208us/step - loss: -2.7080 - acc: 0.1819
Epoch 3/20
21867/21867 [==============================] - 5s 208us/step - loss: -2.1393 - acc: 0.2065
Epoch 4/20
21867/21867 [==============================] - 5s 220us/step - loss: -2.3361 - acc: 0.1676
Epoch 5/20
21867/21867 [==============================] - 5s 223us/step - loss: -2.6815 - acc: 0.2728
Epoch 6/20
21867/21867 [==============================] - 5s 215us/step - loss: -2.2499 - acc: 0.3123
Epoch 7/20
21867/21867 [==============================] - 5s 216us/step - loss: -4.1393 - acc: 0.3156
Epoch 8/20
21867/21867 [==============================] - 5s 216us/step - loss: -2.9147 - acc: 0.3124
Epoch 9/20
21867/21867 [==============================] - 5s 216us/step - loss: -4.1712 - acc: 0.2149
Epoch 10/20
21867/21867 [==============================] - 5s 220us/step - loss: -

In [6]:
path = '/home/ritik/Desktop/music_generation_model/Pop_Music_Midi/'
datax = []
time = 0.0
previous = 0.0
for file in os.listdir(path):
    notes = mido.MidiFile(path + file)
    
    for msg in notes:  
        time += msg.time
        if(msg.type == 'note_on'):
            bit = msg.bytes()
            bit = bit[1:]
            bit.append(msg.channel)
            bit.append(time - previous)
            previous = time
            datax.append(bit)
print(len(datax))

21870


In [7]:
datax = np.array(datax)
mean = np.mean(datax, axis = 0)
std = np.std(datax, axis = 0)

datax = (datax[:] - mean) / std
print(datax, len(datax))

[[ 1.85360243  1.16008307 -1.37072888 -0.5859009 ]
 [-0.00339643  0.3362354   0.13614961 -0.5859009 ]
 [ 0.46085328  0.3362354   0.13614961 -0.5859009 ]
 ...
 [ 0.80904057  0.3362354   0.13614961 -0.5859009 ]
 [ 0.22872843  0.3362354   0.13614961 -0.5859009 ]
 [-1.97645772  0.3362354   1.64302809 -0.5859009 ]] 21870


In [8]:
prediction = []
seed = x_train[12001]
seed = np.expand_dims(seed, axis = 0)

In [13]:
prediction = []
val = np.random.randint(0, x_train.shape[0])
seed = x_train[val]

for ix in range(500):
    seed = np.expand_dims(seed, axis = 0)
    pred = model.predict(seed)
    #print(pred.shape) 
    seed = np.squeeze(seed)
    #print(seed.shape)
    seed = np.concatenate((seed, pred))
    #print(seed.shape)
    seed = seed[1:]
    #print(seed.shape)
    pred = np.squeeze(pred)
    #print(pred.shape)
    prediction.append(pred)

In [10]:
print(prediction)

[array([-1.6305704, -1.066586 ,  1.6528267,  0.9947844], dtype=float32), array([-0.92486566, -0.58362657,  1.2675374 ,  0.3444268 ], dtype=float32), array([ 0.17038032, -0.39577523,  0.80696815, -0.47243905], dtype=float32), array([-0.10233374, -1.4169973 ,  1.7110668 , -0.23608696], dtype=float32), array([-0.0744511, -0.9391769,  1.0448134,  0.1344627], dtype=float32), array([ 0.49372894,  0.10646196, -0.39267516,  0.07256778], dtype=float32), array([ 0.72882736,  0.03391187, -0.389648  , -0.03389842], dtype=float32), array([ 0.22180265, -0.48357555,  0.23705077,  0.12325801], dtype=float32), array([-0.92885315, -0.78892463,  1.2012377 ,  0.4166685 ], dtype=float32), array([-0.8880414 , -0.7480493 ,  1.3065307 ,  0.26868197], dtype=float32), array([-0.09173557, -0.63604337,  0.9138954 , -0.08319281], dtype=float32), array([ 0.15083598, -0.96761006,  1.1780418 , -0.1982867 ], dtype=float32), array([ 0.14086294, -0.8524869 ,  0.90916646,  0.01990817], dtype=float32), array([ 0.40775216,

In [14]:
original_prediction = []
for i in prediction:
    i = (i * std) + mean
    i[0] = int(i[0])
    i[1] = int(i[1])
    i[2] = int(i[2])
    i[3] = i[3]
    original_prediction.append(i)
    
original_prediction = np.array(original_prediction)
print(original_prediction)

[[53.         54.          2.          0.21727402]
 [41.         26.          3.          0.48836245]
 [53.         48.          2.          0.18757911]
 ...
 [59.         51.          2.          0.0921712 ]
 [61.         44.          2.          0.0819506 ]
 [61.         52.          2.          0.13228249]]


In [15]:
midf = mido.MidiFile()
trackss = mido.MidiTrack()
midf.tracks.append(trackss)

for note in original_prediction:
    channel = note[2]
    time = note[2]/0.0125 # to rescale to midi's delta ticks. arbitrary value for now.
    note = note[:2]
    # 147 means note_on
    note = np.insert(note, 0, 147)
    bytes = note.astype(int)
  
    msg = mido.Message.from_bytes(bytes[0:3]) 
    
    msg.time = int(time)
    msg.channel = int(channel)
    trackss.append(msg)
    print (note)

midf.save('/home/ritik/Desktop/music_generation_model/generated_song.midi')

[147.  53.  54.]
[147.  41.  26.]
[147.  53.  48.]
[147.  61.  45.]
[147.  57.  25.]
[147.  61.  55.]
[147.  66.  79.]
[147.  66.  67.]
[147.  58.  56.]
[147.  50.  50.]
[147.  54.  55.]
[147.  61.  54.]
[147.  61.  45.]
[147.  61.  56.]
[147.  64.  74.]
[147.  65.  72.]
[147.  60.  60.]
[147.  51.  51.]
[147.  52.  51.]
[147.  59.  54.]
[147.  61.  47.]
[147.  61.  50.]
[147.  63.  70.]
[147.  65.  75.]
[147.  63.  65.]
[147.  53.  53.]
[147.  50.  50.]
[147.  56.  52.]
[147.  61.  48.]
[147.  61.  45.]
[147.  63.  63.]
[147.  65.  76.]
[147.  64.  70.]
[147.  56.  57.]
[147.  50.  50.]
[147.  53.  51.]
[147.  60.  51.]
[147.  61.  44.]
[147.  62.  54.]
[147.  64.  73.]
[147.  65.  74.]
[147.  61.  62.]
[147.  51.  51.]
[147.  50.  49.]
[147.  57.  52.]
[147.  61.  45.]
[147.  61.  47.]
[147.  63.  68.]
[147.  65.  76.]
[147.  64.  67.]
[147.  54.  54.]
[147.  49.  49.]
[147.  54.  51.]
[147.  60.  48.]
[147.  61.  44.]
[147.  62.  60.]
[147.  65.  75.]
[147.  65.  72.]
[147.  58.  59